In [1]:
file_path = "farmers-protest-tweets-2021-2-4.json"

# Estrategias de Optimización de Velocidad

Los codigos enfocados a la eficiencia de velocidad de procesamiento implementan varias estrategias para procesar eficientemente grandes volúmenes de datos con un uso mínimo de memoria RAM. A continuación, se detallan estas estrategias:

## 1. Uso de `ujson`
- **Implementación**: `import ujson`
- **Beneficio**: `ujson` es significativamente más rápido que el módulo `json` estándar para parsear JSON.

## 2. Carga Completa en Memoria
- **Técnica**: El archivo JSON completo se carga en memoria utilizando `file.readlines()`.
- **Beneficio**: Esto minimiza el tiempo de I/O (entrada/salida) del disco, que suele ser un cuello de botella. Una vez que el archivo está en memoria, todo el procesamiento se realiza en la RAM, que es mucho más rápida.

## 3. Paralelización Completa
- **Explicación**: El código utiliza multiprocessing.Pool para distribuir el procesamiento de líneas entre múltiples núcleos de CPU disponibles. `num_processes = mp.cpu_count() ` garantiza que se aproveche al máximo el hardware disponible.
- **Beneficio**: Esto permite que múltiples líneas se procesen simultáneamente, lo que reduce significativamente el tiempo total de ejecución en comparación con el procesamiento secuencial.

## 4. Extracción directa de datos
- **Implementación**: `date = datetime.fromisoformat(tweet['date'].replace('Z', '+00:00')).date()`
- **Ventaja**: Conversión rápida y directa de la fecha sin procesamiento adicional.

## 5. Uso eficiente de `heapq.nlargest`
- **Código**: `top_10_dates = heapq.nlargest(10, date_tweet_counts.items(), key=lambda x: x[1])`
- **Eficiencia**: Encuentra los top 10 en tiempo O(n log k), donde k = 10, más rápido que ordenar toda la lista.



In [8]:
import cProfile
import pstats
import io

def profile_function(func, file_path):
    pr = cProfile.Profile()
    pr.enable()
    
    result = func(file_path)
    
    pr.disable()
    s = io.StringIO()
    ps = pstats.Stats(pr, stream=s).sort_stats('cumulative')
    ps.print_stats()
    
    print(s.getvalue())
    print(f"Resultado (primeros 10): {result[:10]}")
    
    return result

In [10]:
from q1_time import q1_time
print("Perfilando q1_time:")
result_q1_time = profile_function(q1_time, file_path)


Perfilando q1_time:
         155753 function calls in 1.833 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.080    0.080    1.833    1.833 c:\Users\nicol\OneDrive\Escritorio\LatFirstCommmit\src\q1_time.py:17(q1_time)
        4    0.000    0.000    0.688    0.172 c:\Users\nicol\AppData\Local\Programs\Python\Python311\Lib\threading.py:604(wait)
        4    0.000    0.000    0.688    0.172 c:\Users\nicol\AppData\Local\Programs\Python\Python311\Lib\threading.py:288(wait)
       19    0.688    0.036    0.688    0.036 {method 'acquire' of '_thread.lock' objects}
        1    0.000    0.000    0.688    0.688 c:\Users\nicol\AppData\Local\Programs\Python\Python311\Lib\multiprocessing\pool.py:362(map)
        1    0.000    0.000    0.687    0.687 c:\Users\nicol\AppData\Local\Programs\Python\Python311\Lib\multiprocessing\pool.py:767(get)
        1    0.000    0.000    0.687    0.687 c:\Users\nicol\AppData\Local\Progr

In [9]:
from q2_time import q2_time
print("Perfilando q2_time:")
result_q1_time = profile_function(q2_time, file_path)

Perfilando q2_time:
         190453 function calls (190420 primitive calls) in 1.317 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.005    0.005    1.317    1.317 c:\Users\nicol\OneDrive\Escritorio\LatFirstCommmit\src\q2_time.py:36(q2_time)
        4    0.000    0.000    0.703    0.176 c:\Users\nicol\AppData\Local\Programs\Python\Python311\Lib\threading.py:604(wait)
        1    0.000    0.000    0.703    0.703 c:\Users\nicol\AppData\Local\Programs\Python\Python311\Lib\multiprocessing\pool.py:362(map)
        4    0.000    0.000    0.703    0.176 c:\Users\nicol\AppData\Local\Programs\Python\Python311\Lib\threading.py:288(wait)
       20    0.703    0.035    0.703    0.035 {method 'acquire' of '_thread.lock' objects}
        1    0.000    0.000    0.703    0.703 c:\Users\nicol\AppData\Local\Programs\Python\Python311\Lib\multiprocessing\pool.py:767(get)
        1    0.000    0.000    0.703    0.703 c:\Users\

# Estrategias para Reducir el Consumo de Memoria RAM

Los codigos enfocados a la eficiencia de memoria implementan varias estrategias para procesar eficientemente grandes volúmenes de datos con un uso mínimo de memoria RAM. A continuación, se detallan estas estrategias:

## 1. Procesamiento línea por línea
- **Método**: Lee y procesa el archivo JSON línea por línea.
- **Beneficio**: Solo un tweet está en memoria a la vez, reduciendo significativamente el uso de RAM.

## 2. Estructuras de datos eficientes
- **Uso**: Emplea `Counter` y `defaultdict(Counter)` para contar.
- **Ventaja**: Más eficientes en memoria que diccionarios personalizados para conteo.

## 3. Heap para top 10
- **Implementación**: Mantiene un heap de tamaño fijo (10 elementos) para los top 10 días.
- **Eficiencia**: Evita ordenar toda la lista de días al final, ahorrando memoria.

## 4. Procesamiento incremental
- **Método**: Actualiza conteos y heap mientras lee el archivo.
- **Beneficio**: Evita almacenar todos los datos para procesarlos al final.

## 5. Uso de `ujson`
- **Característica**: Más eficiente en memoria que el módulo `json` estándar.

In [13]:
import ujson
from collections import Counter, defaultdict
from datetime import datetime
from typing import List, Tuple
import heapq
from memory_profiler import profile
from q1_memory import q1_memory

@profile
def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    date_tweet_counts = Counter()
    date_user_counts = defaultdict(lambda: Counter())
    top_dates_heap = []
    
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            tweet = ujson.loads(line)
            date = datetime.fromisoformat(tweet['date'].replace('Z', '+00:00')).date()
            username = tweet['user']['username']
            
            date_tweet_counts[date] += 1
            date_user_counts[date][username] += 1
            
            if len(top_dates_heap) < 10:
                heapq.heappush(top_dates_heap, (date_tweet_counts[date], date))
            elif date_tweet_counts[date] > top_dates_heap[0][0]:
                heapq.heapreplace(top_dates_heap, (date_tweet_counts[date], date))
    
    result = []
    for _, date in sorted(top_dates_heap, reverse=True):
        top_user = date_user_counts[date].most_common(1)[0][0]
        result.append((date, top_user))
    
    return result


result = q1_memory( file_path)
print(f"Resultado (primeros 10): {result[:10]}")

Filename: C:\Users\nicol\AppData\Local\Temp\ipykernel_23876\3829862783.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     9     81.8 MiB     81.8 MiB           1   @profile
    10                                         def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    11     81.8 MiB      0.0 MiB           1       date_tweet_counts = Counter()
    12     84.4 MiB      0.0 MiB          27       date_user_counts = defaultdict(lambda: Counter())
    13     81.8 MiB      0.0 MiB           1       top_dates_heap = []
    14                                             
    15     84.8 MiB      0.0 MiB           2       with open(file_path, 'r', encoding='utf-8') as file:
    16     84.8 MiB      0.3 MiB      117408           for line in file:
    17     84.8 MiB      1.5 MiB      117407               tweet = ujson.loads(line)
    18     84.8 MiB      0.0 MiB      117407               date = datetime.fromisoformat(tweet['date'].replace('Z', '+00:00')).

In [11]:
import ujson as json
from collections import Counter
from typing import List, Tuple
import regex
import heapq
from memory_profiler import profile


@profile
def q2_memory(file_path: str) -> List[Tuple[str, int]]:
    def emoji_process(emoji_counts: Counter) -> Counter:

        """
        Función para procesar y combinar componentes de emojis, especialmente los que forman
        banderas, en emojis completos. También maneja otros tipos de emojis, excluyendo 
        modificadores de tono de piel.
        """

        combined = Counter()
        flag_components = {}
        for emoji, count in emoji_counts.items():
             # Detecta y combina los componentes individuales de las banderas (letras de país)
            if len(emoji) == 1 and '\U0001F1E6' <= emoji <= '\U0001F1FF':
                if flag_components:
                    prev_emoji = list(flag_components.keys())[0]
                    combined_emoji = prev_emoji + emoji
                     # Combina los componentes en un solo emoji de bandera
                    combined[combined_emoji] = min(count, emoji_counts[prev_emoji])
                    flag_components.clear()
                else:
                    flag_components[emoji] = count
            elif emoji not in ['\U0001F3FB', '\U0001F3FC', '\U0001F3FD', '\U0001F3FE', '\U0001F3FF']:  # Excluir modificadores de tono de piel
                combined[emoji] = count

        combined.update(flag_components) # Agrega cualquier componente de bandera que quedó sin combinar
        
        return combined

    emoji_counter = Counter()
    emoji_pattern = regex.compile(r'\p{Emoji_Presentation}|\p{Emoji}\uFE0F')
    heap = []
    tweets_processed = 0
    tweets_with_emojis = 0
    
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                tweet = json.loads(line)
                content = tweet.get('content', '')
                
                if not content:
                    continue
                
                emojis = emoji_pattern.findall(content)
                
                if emojis:
                    tweets_with_emojis += 1
                    for emoji in emojis:
                        emoji_counter[emoji] += 1
                        
                        # Actualizar el heap para mantener solo el top 10 emojis
                        if len(heap) < 10:
                            heapq.heappush(heap, (emoji_counter[emoji], emoji))
                        else:
                            heapq.heappushpop(heap, (emoji_counter[emoji], emoji))
            
            except json.JSONDecodeError:
                print(f"Error al decodificar JSON en la línea {tweets_processed}")
            except Exception as e:
                print(f"Error inesperado en la línea {tweets_processed}: {str(e)}")
    
    combined_counter = emoji_process(emoji_counter)
    
    # Obtener los top 10 emojis más frecuentes desde el heap
    top_10_emojis = heapq.nlargest(10, combined_counter.items(), key=lambda x: x[1])
    
    return top_10_emojis

result = q2_memory( file_path)
print(f"Resultado (primeros 10): {result[:10]}")

Filename: C:\Users\nicol\AppData\Local\Temp\ipykernel_23876\1966117880.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     9     84.6 MiB     84.6 MiB           1   @profile
    10                                         def q2_memory(file_path: str) -> List[Tuple[str, int]]:
    11     84.9 MiB      0.0 MiB           2       def emoji_process(emoji_counts: Counter) -> Counter:
    12                                         
    13                                                 """
    14                                                 Función para procesar y combinar componentes de emojis, especialmente los que forman
    15                                                 banderas, en emojis completos. También maneja otros tipos de emojis, excluyendo 
    16                                                 modificadores de tono de piel.
    17                                                 """
    18                                         
    19     84.9 MiB     